# Training Yolo v5 using custom data annotation

After successful data prepration, we are trying to detect the Hexbugs using `Yolo v5`. We are doing this using the `Pytorch` library. The original data annotations that we were provided with, contained only the `(x, y)` position of the Hexbugs' heads' position. To train a `Yolo` model, we need to do extra data annotations to obtain the bouding boxes. To do so, we used [Roboflow](https://roboflow.com/). Roboflow is an awesome platform for data annotation which allows teams to work on uploaded data in groups. After annotating the frames, we exported the dataset as a `Yolo v5 dataset`. Please mention that the original libraries are implemented in `Pytorch`. Therefore, make sure that you have access to proper GPUs to run the code.

### Check Nvidia driver

In [ ]:
!nvidia-smi

### Required imports

* `os` is required to manage directories.
* `ultralytics` is essential to load `Yolo`
* `roboflow` is required to download the dataset from the server.

In [ ]:
# Pip install method (recommended)
!pip install ultralytics==8.0.20
!pip install roboflow --quiet

import os
import glob

from credentials import YOUR_API_KEY
from IPython import display
from IPython.display import display, Image
import ultralytics
from ultralytics import YOLO
from roboflow import Roboflow

ultralytics.checks()

In [ ]:
# Setting Home variable to work easier with the file paths later.
HOME = os.getcwd()
print(HOME)

### Resetting environment

In [ ]:
# Removing already existing `datasets` directory
!mkdir {HOME}/datasets
%cd {HOME}/datasets

### Downloading dataset from `Roboflow`

In [ ]:
# Using `Roboflow` library to access and downloading the dataset.

rf = Roboflow(api_key=YOUR_API_KEY)
project = rf.workspace("hexbugs").project("bugbusters")
dataset = project.version(4).download("yolov5")

In [ ]:
%cd {HOME}
# using the already implemented `yolo` package to perform a training task.
# It'll download the `Yolov8s.pt` mdoel for reference and use its already trained weights for transfer learning.
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=35 imgsz=800 plots=True

In [ ]:
!ls {HOME}/runs/detect/train/

### Results Visualization
Most of the times, it's a great idea to create a confusion matrix to understand the performance of the model. Here, we plot the confusion matrix, but it only has one class, `HexBug`.

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

### Plotting metrics
We plot `loss`, `recall`, and `mAP` as metrics to analyze the model performance better.

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

### Plotting results over validation set

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

### Performing validation over the validation set

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

### Performing detection over test set

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

### Plotting the detection result over unseen dataset

In [ ]:
for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

### Result

Now we have a fully trained `Yolo` model that performs quite good in detecting Hexbugs.
Now it's time to use this model to detect Hexbugs in the frames and get rid of void areas.
The best model is now located at `runs/detect/train/weights/best.pt`.